In [51]:
import pandas as pd
import requests
import time
import os
import dotenv
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("WEATHER_API_KEY")
df = pd.read_csv("data\\crime_within_station_walksheds.csv")

In [52]:
def fetch_daily_weather(lat, lon, date_str, unitGroup="us"):
    """
    Returns a dict with daily summary weather for a single (lat, lon, date).
    """
    base = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline"
    location = f"{lat},{lon}"
    url = f"{base}/{location}/{date_str}"

    params = {
        "key": API_KEY,
        "contentType": "json",
        "unitGroup": unitGroup,
        "include": "days",
        # Request only what you need (plus datetime for safety)
        "elements": ",".join([
            "datetime",
            "tempmax",
            "tempmin",
            "temp",
            "precip",
            "precipcover",
            "cloudcover",
            "humidity",
            "windspeed",
            "windgust",
            "visibility",
            "snow",
            "conditions"
])

    }

    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    data = r.json()

    # Daily summary lives in data["days"][0] for a single-day query
    day = data["days"][0]
    return {
        "DATE_INCIDENT_BEGAN": day.get("datetime"),
        "tempmax": day.get("tempmax"),
        "tempmin": day.get("tempmin"),
        "temp": day.get("temp"),
        "precip": day.get("precip"),
        "precipcover": day.get("precipcover"),
        "cloudcover": day.get("cloudcover"),
        "humidity": day.get("humidity"),
        "windspeed": day.get("windspeed"),
        "windgust": day.get("windgust"),
        "visibility": day.get("visibility"),
        "snow": day.get("snow"),
        "conditions": day.get("conditions"),
    }

In [53]:
df["DATE_INCIDENT_BEGAN"] = pd.to_datetime(df["DATE_INCIDENT_BEGAN"]).dt.strftime("%Y-%m-%d")

# Optional: reduce unique calls by rounding coords
df["LATITUDE_PUBLIC"] = df["LATITUDE_PUBLIC"].round(2)
df["LONGITUDE_PUBLIC"] = df["LONGITUDE_PUBLIC"].round(2)

keys = df[["DATE_INCIDENT_BEGAN", "LATITUDE_PUBLIC", "LONGITUDE_PUBLIC"]].drop_duplicates()

In [54]:
weather_rows = []
for i, row in keys.iterrows():
    try:
        w = fetch_daily_weather(row["LATITUDE_PUBLIC"], row["LONGITUDE_PUBLIC"], row["DATE_INCIDENT_BEGAN"])
        w["LATITUDE_PUBLIC"] = row["LATITUDE_PUBLIC"]
        w["LONGITUDE_PUBLIC"] = row["LONGITUDE_PUBLIC"]
        weather_rows.append(w)
    except Exception as e:
        # Keep going; log failures
        print(e)
        weather_rows.append({
            "DATE_INCIDENT_BEGAN": row["DATE_INCIDENT_BEGAN"],
            "LATITUDE_PUBLIC": row["LATITUDE_PUBLIC"],
            "LONGITUDE_PUBLIC": row["LONGITUDE_PUBLIC"],
            "tempmax": None,
            "tempmin": None,
            "temp": None,
            "precip": None,
            "precipcover": None,
            "cloudcover": None,
            "humidity": None,
            "windspeed": None,
            "windgust": None,
            "visibility": None,
            "snow": None,
            "conditions": None,
            "error": str(e)[:200]
        })

    # simple rate-limiting to be safe
    time.sleep(0.1)
    if i % 1000 == 0:
        print(i)

weather_df = pd.DataFrame(weather_rows)

0
1000
3000
5000
8000
12000
14000
17000
19000
400 Client Error:  for url: https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/35.23,-80.84/0201-08-20?key=UFLSNGSPH5C5W93V8ETBQ26A5&contentType=json&unitGroup=us&include=days&elements=datetime%2Ctempmax%2Ctempmin%2Ctemp%2Cprecip%2Cprecipcover%2Ccloudcover%2Chumidity%2Cwindspeed%2Cwindgust%2Cvisibility%2Csnow%2Cconditions
24000
26000
400 Client Error:  for url: https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/35.22,-80.84/1949-01-01?key=UFLSNGSPH5C5W93V8ETBQ26A5&contentType=json&unitGroup=us&include=days&elements=datetime%2Ctempmax%2Ctempmin%2Ctemp%2Cprecip%2Cprecipcover%2Ccloudcover%2Chumidity%2Cwindspeed%2Cwindgust%2Cvisibility%2Csnow%2Cconditions
38000


In [55]:
df_complete = df.merge(weather_df, on=["DATE_INCIDENT_BEGAN", "LATITUDE_PUBLIC", "LONGITUDE_PUBLIC"], how="left")

In [56]:
df_complete.isna().sum()

LOCATION                         0
ZIP                              0
LATITUDE_PUBLIC                  0
LONGITUDE_PUBLIC                 0
CMPD_PATROL_DIVISION           216
NPA                              0
DATE_INCIDENT_BEGAN              0
LOCATION_TYPE_DESCRIPTION        0
PLACE_TYPE_DESCRIPTION           0
PLACE_DETAIL_DESCRIPTION         0
HIGHEST_NIBRS_DESCRIPTION        0
cluster                          0
cluster_title                    0
geometry                         0
index_right                      0
name                             0
lat                              0
lon                              0
tempmax                          3
tempmin                          3
temp                             3
precip                           3
precipcover                      3
cloudcover                       3
humidity                         3
windspeed                        3
windgust                      8966
visibility                       3
snow                

In [57]:
df_complete.head(20)

,LOCATION,ZIP,LATITUDE_PUBLIC,LONGITUDE_PUBLIC,CMPD_PATROL_DIVISION,NPA,DATE_INCIDENT_BEGAN,LOCATION_TYPE_DESCRIPTION,PLACE_TYPE_DESCRIPTION,PLACE_DETAIL_DESCRIPTION,...,precip,precipcover,cloudcover,humidity,windspeed,windgust,visibility,snow,conditions,error
0,200 E TRADE ST,28202,35.23,-80.84,Central,476,2018-11-17,Outdoors,Open Area,Street/Highway,...,0.000,0.00,15.8,70.1,7.1,NaN,9.4,0.0,Clear,NaN
1,200 E TRADE ST,28202,35.23,-80.84,Central,476,2018-11-17,Outdoors,Open Area,Street/Highway,...,0.000,0.00,15.8,70.1,7.1,NaN,9.4,0.0,Clear,NaN
2,200 E TRADE ST,28202,35.23,-80.84,Central,476,2018-11-17,Outdoors,Open Area,Street/Highway,...,0.000,0.00,15.8,70.1,7.1,NaN,9.4,0.0,Clear,NaN
3,400 S DAVIDSON ST,28204,35.22,-80.84,Central,476,2018-12-21,Outdoors,Open Area,Street/Highway,...,0.167,50.00,94.7,88.1,18.7,33.6,8.5,0.0,"Rain, Overcast",NaN
4,400 S DAVIDSON ST,28204,35.22,-80.84,Central,476,2018-12-21,Outdoors,Open Area,Street/Highway,...,0.167,50.00,94.7,88.1,18.7,33.6,8.5,0.0,"Rain, Overcast",NaN
5,600 E TRADE ST,28202,35.22,-80.84,Central,476,2020-04-10,Other,Open Area,Cyberspace,...,0.000,0.00,19.7,28.1,18.8,31.8,9.9,0.0,Clear,NaN
6,600 E TRADE ST,28202,35.22,-80.84,Central,476,2020-04-10,Other,Open Area,Cyberspace,...,0.000,0.00,19.7,28.1,18.8,31.8,9.9,0.0,Clear,NaN
7,400 N TRYON ST,28202,35.23,-80.84,Central,476,2017-06-25,Outdoors,Commercial Place,Restaurant/Diner/Coffee Shop,...,0.000,0.00,55.5,73.3,9.8,17.2,9.9,0.0,Partially cloudy,NaN
8,400 N TRYON ST,28202,35.23,-80.84,Central,476,2017-06-25,Outdoors,Commercial Place,Restaurant/Diner/Coffee Shop,...,0.000,0.00,55.5,73.3,9.8,17.2,9.9,0.0,Partially cloudy,NaN
9,200 EAST BV,28203,35.21,-80.86,Central,3,2021-05-24,Indoors,Retail,Convenience Store,...,0.000,0.00,51.8,54.7,8.5,NaN,9.3,0.0,Partially cloudy,NaN


In [61]:
df_complete['snow'] = df_complete['snow'].fillna(0)
df_complete.dropna(subset=['temp'], inplace=True)
df_complete.drop(columns=['error', 'windgust'], inplace=True)
df_complete.isna().sum()

LOCATION                       0
ZIP                            0
LATITUDE_PUBLIC                0
LONGITUDE_PUBLIC               0
CMPD_PATROL_DIVISION         216
NPA                            0
DATE_INCIDENT_BEGAN            0
LOCATION_TYPE_DESCRIPTION      0
PLACE_TYPE_DESCRIPTION         0
PLACE_DETAIL_DESCRIPTION       0
HIGHEST_NIBRS_DESCRIPTION      0
cluster                        0
cluster_title                  0
geometry                       0
index_right                    0
name                           0
lat                            0
lon                            0
tempmax                        0
tempmin                        0
temp                           0
precip                         0
precipcover                    0
cloudcover                     0
humidity                       0
windspeed                      0
visibility                     0
snow                           0
conditions                     0
dtype: int64

In [62]:
df_complete.to_csv("data\\crime_with_weather.csv", index=False)